In [88]:
import pandas as pd
import glob
import os

m_store = pd.read_csv("d:\\data_analysis_lab\\ML_100\\data\\m_store.csv")
m_area = pd.read_csv("d:\\data_analysis_lab\\ML_100\\data\\m_area.csv")


current_dir = os.getcwd()
# 찾을 파일 패턴 만들기
tbl_order_file = os.path.join(current_dir + "\\data\\", 'tbl_order_*.csv')
# 해당하는 패턴과 일치하는 모든 파일 경로를 리스트로 변환
tbl_order_files = glob.glob(tbl_order_file)
order_all  = pd.DataFrame()
for file in tbl_order_files :
    order_tmp = pd.read_csv(file)
    order_all = pd.concat([order_all, order_tmp], ignore_index=True)
    order_all.head()
    


In [89]:
order_all.head()

,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status
0,34104383,11,C65806632,57,61573513,2021-04-01 11:00,2021-04-01 11:39,1,28270,1
1,70652318,59,C09760173,37,54068709,2021-04-01 11:00,2021-04-01 11:34,0,28270,2
2,71640388,195,C61227084,17,93678366,2021-04-01 11:00,2021-04-01 11:54,0,26470,9
3,75673365,127,C64119972,17,5287952,2021-04-01 11:00,2021-04-01 11:17,0,23080,2
4,9077529,174,C10231192,18,18248867,2021-04-01 11:00,2021-04-01 11:35,0,46920,2


In [90]:
# 유지 보수용 매장 데이터 삭제  
order_all = order_all.loc[order_all['store_id'] != 999]

order_all = pd.merge(order_all, m_store, on="store_id", how='left')
order_all = pd.merge(order_all, m_area, on="area_cd", how='left')

In [91]:
# 마스터에 없는 코드에 대응한 문자열 설정

order_all.loc[order_all['takeout_flag'] == 0, 'takeout_name'] = "delivery"
order_all.loc[order_all['takeout_flag'] == 1, 'takeout_name'] = "takeout"


order_all.loc[order_all['status'] == 0, 'status_name'] = '주문접수'
order_all.loc[order_all['status'] == 1, 'status_name'] = '결제완료'
order_all.loc[order_all['status'] == 2, 'status_name'] = '배달완료'
order_all.loc[order_all['status'] == 3, 'status_name'] = '주문취소'
print(order_all.head())

   order_id  store_id customer_id  coupon_cd  sales_detail_id  \
0  34104383        11   C65806632         57         61573513   
1  70652318        59   C09760173         37         54068709   
2  71640388       195   C61227084         17         93678366   
3  75673365       127   C64119972         17          5287952   
4   9077529       174   C10231192         18         18248867   

  order_accept_date    delivered_date  takeout_flag  total_amount  status  \
0  2021-04-01 11:00  2021-04-01 11:39             1         28270       1   
1  2021-04-01 11:00  2021-04-01 11:34             0         28270       2   
2  2021-04-01 11:00  2021-04-01 11:54             0         26470       9   
3  2021-04-01 11:00  2021-04-01 11:17             0         23080       2   
4  2021-04-01 11:00  2021-04-01 11:35             0         46920       2   

  store_name area_cd wide_area narrow_area takeout_name status_name  
0       자양로점      SL        서울          서울      takeout        결제완료  
1     

In [92]:
import openpyxl

# 엑셀파일 만들기 
wb = openpyxl.Workbook()
ws = wb['Sheet']
ws.cell(1,1).value = "파일 쓰기 테스트입니다"
wb.save("d:\\data_analysis_lab\\ML_100\\data\\test.xlsx")
wb.close()

In [93]:
wb = openpyxl.load_workbook("d:\\data_analysis_lab\\ML_100\\data\\test.xlsx", read_only=True)
ws = wb['Sheet']
print(ws.cell(1,1).value)
wb.close()

파일 쓰기 테스트입니다


In [94]:
# 테스트 데이터 준비 

store_id =1 
store_df = order_all.loc[order_all['store_id'] == store_id].copy()
store_name = store_df['store_name'].unique()[0]
store_sales_total = store_df.loc[store_df['status'].isin([1,2])]['total_amount'].sum()
store_sales_takeout = store_df.loc[store_df['status'] == 1]['total_amount'].sum()
store_sales_delivery = store_df.loc[store_df['status'] == 2]['total_amount'].sum()
print(f'매출확인 {store_sales_total} =', f'{store_sales_takeout + store_sales_delivery}')

ouput_df = store_df[['order_accept_date', 'customer_id', 'total_amount', 'takeout_name', 'status_name']]
ouput_df.head()

매출확인 90101050 = 90101050


,order_accept_date,customer_id,total_amount,takeout_name,status_name
48,2021-04-01 11:03,C81203759,30500,takeout,결제완료
435,2021-04-01 11:32,C93103282,26150,takeout,결제완료
627,2021-04-01 11:50,C33623566,41440,delivery,NaN
642,2021-04-01 11:51,C64109769,33000,delivery,배달완료
976,2021-04-01 12:21,C46929234,26030,delivery,NaN


In [95]:
from openpyxl.utils.dataframe import dataframe_to_rows

store_title = f'{store_id}_{store_name}'

wb = openpyxl.Workbook()
ws = wb.active
ws.title = store_title

ws.cell(1,1).value = f'{store_title} 매출 데이터'

rows = dataframe_to_rows(ouput_df, index=False, header=True)

row_start = 3
col_start = 2

for row_no, row in enumerate(rows, row_start) :
    for col_no, value in enumerate(row, col_start) : 
        ws.cell(row_no, col_no).value = value

filename = f'd:\\data_analysis_lab\\ML_100\\data\\{store_title}.xlsx'
wb.save(filename)
wb.close()

In [96]:
from openpyxl.styles import PatternFill, Border, Side, Font

openpyxl.load_workbook(filename)
ws = wb[store_title]

side = Side(style='thin', color='008080')
border = Border(top=side, bottom=side, left=side, right=side)

for row in ws :
    for cell in row:
        if ws[cell.coordinate].value :
            ws[cell.coordinate].border =  border

In [97]:
ws.cell(1,1).font = Font(bold=True, color="008080")

cell = ws.cell(3,2)
cell.fill = PatternFill(patternType='solid', fgColor='008080')
cell.value = '주문 접수 일시'
cell.font = Font(bold=True, color='FFFFFF')

cell = ws.cell(3,3)
cell.fill = PatternFill(patternType='solid', fgColor='008080')
cell.value = '고객 ID'
cell.font = Font(bold=True, color='FFFFFF')

cell = ws.cell(3,4)
cell.fill = PatternFill(patternType='solid', fgColor='008080')
cell.value = '수입 총액'
cell.font = Font(bold=True, color='FFFFFF')

cell = ws.cell(3,5)
cell.fill = PatternFill(patternType='solid', fgColor='008080')
cell.value = '주문 타입'
cell.font = Font(bold=True, color='FFFFFF')

cell = ws.cell(3,6)
cell.fill = PatternFill(patternType='solid', fgColor='008080')
cell.value = '주문 상태'
cell.font = Font(bold=True, color='FFFFFF')

ws.column_dimensions['A'].width = 20
ws.column_dimensions['B'].width = 20
ws.column_dimensions['C'].width = 12
ws.column_dimensions['D'].width = 12
ws.column_dimensions['E'].width = 12
ws.column_dimensions['F'].width = 12


print("파일에 저장")
wb.save(filename)
wb.close()

파일에 저장


In [98]:
# 매출 이외의 데이터 출력

def cal_dalta(t):
    t1, t2 = t
    delta = t2 - t1
    return delta.total_seconds() / 60


store_df.loc[:, 'order_accept_datetime'] = pd.to_datetime(store_df['order_accept_date'])
store_df.loc[:, 'delivered_datetime'] = pd.to_datetime(store_df['delivered_date'])
store_df.loc[:, 'delta'] = store_df[['order_accept_datetime', 'delivered_datetime']].apply(cal_dalta, axis=1)

delivery_time = store_df.groupby(['store_id'])['delta'].describe()
delivery_time

,count,mean,std,min,25%,50%,75%,max
store_id,,,,,,,,
1,3553.0,34.46327,14.502363,10.0,22.0,34.0,47.0,59.0


In [99]:
openpyxl.load_workbook(filename)
ws = wb[store_title]

cell = ws.cell(1, 7)
cell.value = f'배달시간'
cell.font = Font(bold=True, color = '008080')

rows = dataframe_to_rows(delivery_time, index=False, header=True)

In [100]:
# 표 삽입 위치 

row_start = 3
col_start = 8

for row_no, row in enumerate(rows, row_start) :
    for col_no, value in enumerate(row, col_start) :
        cell = ws.cell(row_no, col_no)
        cell.value = value
        cell.border = border
        if row_no == row_start :
            cell.fill = PatternFill(patternType='solid', fgColor='008080')
            cell.font = Font(bold=True, color='FFFFFF')
            
filename = f'd:\\data_analysis_lab\\ML_100\\data\\{store_title}.xlsx'
wb.save(filename)
wb.close()

In [101]:
# 문제가 있는 위치를 빨간색으로 출력 

openpyxl.load_workbook(filename)
ws = wb[store_title]

rows = dataframe_to_rows(ouput_df, index=False, header=True)

row_start = 3
col_start = 2

for row_no, row in enumerate(rows, row_start) :
    if row_no == row_start : 
        continue
    for col_no, value in enumerate(row, col_start) :
        ws.cell(row_no, col_no).value = value
        if value == '결제완료':
            ws.cell(row_no, col_no).font = Font(bold=True, color='FF0000')

filename = f'd:\\data_analysis_lab\\ML_100\\data\\{store_title}.xlsx'
wb.save(filename)
wb.close()

In [102]:
openpyxl.load_workbook(filename)

ws = wb[store_title]

cell = ws.cell(7,7)
cell.value = '집계'
cell.font = Font(bold=True, color="008080")

cell = ws.cell(8,8)
cell.value = "데이터 총액"
cell.font = Font(bold=True, color="008080")

cell = ws.cell(8,10)
cell.value = f'= SUM(D4:D{ws.max_row})'

cell = ws.cell(9,8)
cell.value = "내 결제 완료 금액"
cell.font = Font(bold=True)

cell = ws.cell(9,10)
cell.value = f'= SUMIF(F4:F{ws.max_row}, "<>주문취소", D4:D{ws.max_row})'

cell = ws.cell(10,8)
cell.value = "내 주문취소 금액"
cell.font = Font(bold=True)

cell = ws.cell(10,10)
cell.value = cell.value = f'= SUMIF(F4:F{ws.max_row}, "=주문취소", D4:D{ws.max_row})'


filename = f'd:\\data_analysis_lab\\ML_100\\data\\{store_title}.xlsx'
wb.save(filename)
wb.close()

In [103]:
def calc_delta(t) :
    t1, t2 = t
    delta =  t2 - t1
    return delta.total_seconds() / 60

In [104]:
# 주문 취소율 데이터 준비 
cancel_df = pd.DataFrame()
cancel_cnt = order_all.loc[order_all['status'] == 9].groupby(['store_id'])['store_id'].count()
order_cnt = order_all.loc[order_all['status'].isin([1,2,9])].groupby(['store_id'])['store_id'].count()

cancel_rate = (cancel_cnt / order_cnt) * 100
cancel_df['cancel_rate'] = cancel_rate
cancel_df = pd.merge(cancel_df, m_store, on='store_id', how='left')
cancel_rank = cancel_df.sort_values('cancel_rate', ascending=True).reset_index()


# 주문 취소율 순위와 매장 순위 확인 함수 
def check_store_cancel_rank(trg_id) :
    tmp = cancel_rank.loc[cancel_rank['store_id'] == trg_id].index + 1
    return tmp[0]

In [105]:
def get_area_df(trg_id) :
    area_df = pd.DataFrame()
    area_df = order_all.loc[order_all['area_cd'] == store_df['area_cd'].unique()[0]]
    area_df = area_df.loc[area_df['status'].isin([1,2])]
    return area_df

def get_area_rank_df(trg_id) :
    area_df = get_area_df(trg_id)
    area_rank = area_df.groupby(['store_id'])['total_amount'].sum().sort_values(ascending=True)
    area_rank = pd.merge(area_rank, m_store, on="store_id", how="left")
    return area_rank
    
def check_store_sales_rank(trg_id) :
    area_rank = get_area_rank_df(trg_id)
    tmp = area_rank.loc[area_rank['store_id'] == trg_id].index + 1
    return tmp[0]

In [106]:
def make_store_daliy(trg_id) :
    tmp_store_df = order_all.loc[(order_all['store_id'] == trg_id) & (order_all['status'].isin([1,2]))]
    tmp = tmp_store_df[['order_accept_date', 'total_amount']].copy()
    tmp.loc[:,'order_accept_date'] = pd.to_datetime(tmp['order_accept_date'])
    tmp.set_index('order_accept_date', inplace=True)
    tmp = tmp.resample('D').sum().reset_index()
    return tmp

In [107]:
def get_area_delivery(trg_id) : 
    # 해당 점포가 속한 지역별 데이터의 배달 완료까지의 시간 순위
    area_delivery = pd.DataFrame()
    area_df = get_area_df(trg_id)
    area_delivery = area_df.loc[area_df['status'] == 2].copy()
    area_delivery.loc[:, 'order_accept_datetime'] = pd.to_datetime(area_delivery['order_accept_date'])
    area_delivery.loc[:, 'delivered_datetime'] = pd.to_datetime(area_delivery['delivered_date'])
    area_delivery.loc[:, 'delta'] = area_delivery[['order_accept_datetime','delivered_datetime']].apply(calc_delta, axis=1)
    return area_delivery

In [108]:
area_df = get_area_df(store_id)
area_df.head()


,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,store_name,area_cd,wide_area,narrow_area,takeout_name,status_name
0,34104383,11,C65806632,57,61573513,2021-04-01 11:00,2021-04-01 11:39,1,28270,1,자양로점,SL,서울,서울,takeout,결제완료
1,70652318,59,C09760173,37,54068709,2021-04-01 11:00,2021-04-01 11:34,0,28270,2,가마산로2점,SL,서울,서울,delivery,배달완료
7,10290387,30,C17281363,75,81945254,2021-04-01 11:00,2021-04-01 11:16,1,22380,1,마들로점,SL,서울,서울,takeout,결제완료
8,54821099,9,C16681192,90,9773815,2021-04-01 11:00,2021-04-01 11:35,0,23080,2,남부순환로점,SL,서울,서울,delivery,배달완료
10,71238056,79,C81534225,30,18403035,2021-04-01 11:00,2021-04-01 11:16,0,38650,2,종로2점,SL,서울,서울,delivery,배달완료


In [109]:
def get_area_delivery_rank_df(trg_id) :
    area_delivery = get_area_delivery(trg_id)
    area_delivery_rank = area_delivery.groupby(['store_id'])['delta'].mean().sort_values()
    area_delivery_rank = pd.merge(area_delivery_rank, m_store, on='store_id', how='left')
    return area_delivery_rank

def check_store_delivery_rank(trg_id) :
    area_delivery_rank = get_area_delivery_rank_df(trg_id)
    tmp = area_delivery_rank.loc[area_delivery_rank['store_id'] == trg_id].index + 1
    return tmp[0]

In [110]:
def data_sheet_output(trg_wb, sheet_name, target_df, index_flg) :
    ws = trg_wb.create_sheet(title = sheet_name)
    rows = dataframe_to_rows(target_df, index= index_flg, header=True)
    
row_start = 3
col_start = 2

for row_no, row in enumerate(rows, row_start) :
    for col_no, value in enumerate(row, col_start) : 
        ws.cell(row_no, col_no).value = value
        ws.sheet_state = 'hidden'
        
        
        
def make_data_sheet(trg_id, trg_st_df, target_folder) :
    target_daliy = make_store_daliy(trg_id)
    store_name = trg_st_df['store_name'].unique()[0]
    
    store_title = f'{trg_id}_{store_name}'
    
    wb = openpyxl.Workbook()
    
    #지역 취소 순위
    data_sheet_output(wb, 'Data_CancelRank', cancel_rank, False)
    
    #지역 매출 순위
    data_sheet_output(wb, 'Data_AreaRank', get_area_delivery_rank_df(trg_id), False)
    
    #지역 배달 완료 시간 순위
    data_sheet_output(wb, 'Data_DeliveryRank', get_area_delivery_rank_df(trg_id), False)
    
    # 해당 매장의 일단위 매출 데이터 
    data_sheet_output(wb, 'Data_Target_Daliy', target_daliy, False)
    
    filname = os.path.join(target_folder, f'd:\\data_analysis_lab\\ML_100\\data\\{store_title}.xlsx')
    wb.save(filename)
    wb.close()
    
    return filename

In [111]:
filename_store = make_data_sheet(store_id, store_df, '')

d:\anaconda\Lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


In [112]:
from pyecharts.charts import Line as Series
from openpyxl.chart import LineChart, Reference, Series

def make_summary_sheet(trg_id, storename, trgfile) :
    target_cancel_rank = check_store_cancel_rank(trg_id)
    target_sales_rank = check_store_sales_rank(trg_id)
    target_delivery_rank = check_store_delivery_rank(trg_id)
    
    wb = openpyxl.load_workbook(trgfile)
    ws = wb.active
    ws.title = "요약보고서"
    
    cell = ws.cell(1,1)
    
    cell.value = f'{storename} 요약보고서(4월~6월)'
    cell.font = Font(bold=True, color='008080', size=20)
    
    # 매출순위표시 
    tmpWs = wb['Data_Target_Daliy']
    cell = ws.cell(3,2)
    cell.value = "매장 매출액"
    cell.font = Font(bold=True, color='008080', size=16)
    
    #셀병합
    ws.merge_cells('E3:F3')
    
    cell = ws.cell(3,5)
    cell.value = f'=SUM({tmpWs.title}!B2:B{tmpWs.max_row})'
    cell.font = Font(bold=True, color='0080FF', size=16)
    cell.number_format = "#,##0"
    
        
    cell = ws.cell(4,2)
    cell.value = '매장매출순위'
    cell.font = Font(bold=True, color='0080FF', size=16)

    cell = ws.cell(4,5)
    cell.value = f'{len(m_store)}개 매장 중 {target_sales_rank}]'
    cell.font = Font(bold=True, color='0080FF', size=16)
    
    
    # 그래프용 참조 데이터 지정
    refy = Reference(tmpWs, min_col=2, min_row=2, max_col=2, max_row=tmpWs.max_row)
    
    series = Series(refy, title="매출액")
    
    # chart
    chart = LineChart()
    chart.title = "기간 매출액(일별)"
    chart.x_axis.title = "건수"
    chart.y_axis.title = "매출액"
    chart.height = 10 
    chart.width = 15
    chart.series.append(series)
    
    
    ws.add_chart(chart, 'B6')
    
    tmpWs = wb['Data_AreaRank']
    
    cell = ws.cell(4,10)
    cell.value = '지역 매장 매출 정보'
    cell.font = Font(bold=True, color="008080", size=16)
    
    cell = ws.cell(5,11)
    cell.value = "최고금액"
    
    cell = ws.cell(5,12)
    cell.value = f'=MAX({tmpWs.title}!B2:B{tmpWs.max_row})'
    cell.number_format = "#,##0"    
    
    cell = ws.cell(6,11)
    cell.value = '최저금액'
    
    cell = ws.cell(6,12)
    cell.value = f'=MIN({tmpWs.title}!B2:B{tmpWs.max_row})'
    cell.number_format = "#,##0"    
    
    cell = ws.cell(7, 11)
    cell.value = f'=AVERAGE({tmpWs.title}!B2:B{tmpWs.max_row})'
    cell.number_format = "#,##0" 
    
    # 주문 취소율 표시 
    cell = ws.cell(11,10)
    cell.value = '주문 취소 순위'
    cell.font = Font(bold=True, color="008080", size=16)
    
    cell = ws.cell(12,11)
    cell.value = f'{len(m_store)}개 매장 중 {target_cancel_rank}위'
    cell.font = Font(bold=True, color='0080FF', size=16)
    
    tmpWs = wb['Data_CancelRank']
    
    cell = ws.cell(13, 11)
    cell.value = '지역평균'
    
    
    
    cell = ws.cell(13, 12)
    cell.value = f'=AVERAGE({tmpWs.title}!B2:B{tmpWs.max_row})'
    cell.number_format = "0.00"
    
    # 배달 시간 순위 표시 
    cell = ws.cell(15, 10)
    cell.value = '배달 완료 시간 순위'
    cell.font = Font(bold=True, color="008080", size=16)
    
    cell = ws.cell(16, 11)
    cell.value = f'{len(m_store)}개 매장 중 {target_cancel_rank}위'
    cell.font = Font(bold=True, color='0080FF', size=16)
    
    
    tmpWs = wb['Data_DeliveryRank']
    
    cell = ws.cell(17,11)
    cell.value = '지역평균'
    cell = ws.cell(17,12)
    cell.value = f'=AVERAGE({tmpWs.title}!B2:B{tmpWs.max_row})'
    cell.number_format = '0.00'
    
    wb.save(trgfile)
    wb.close()

In [114]:
make_summary_sheet(store_id, store_name, filename_store)

In [116]:
os.makedirs('output', exist_ok=True)

for store in m_store['store_id'].tolist() : 
    if store != 999:
        store_df = order_all.loc[order_all['store_id']==store]
        store_name = m_store.loc[m_store['store_id'] == store]['store_name']
        print(store_name)
        
        tmp_file_name = make_data_sheet(store, store_df, 'output')
        make_summary_sheet(store, store_name.values[0], tmp_file_name)

print("출력을 완료했습니다")

0    삼일대로점
Name: store_name, dtype: object


d:\anaconda\Lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


1    세종대로점
Name: store_name, dtype: object


d:\anaconda\Lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


2    무교로점
Name: store_name, dtype: object


...

194    화중로점
Name: store_name, dtype: object


d:\anaconda\Lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


195    석봉로점
Name: store_name, dtype: object


d:\anaconda\Lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


출력을 완료했습니다


In [ ]:
"chapter 5부터 시작"